<a href="https://colab.research.google.com/github/Mananpatel25/nlp-assignments/blob/main/Bloomz3b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing and checking cuda
import os
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    gpu_info = !nvidia-smi --query-gpu=name,memory.total,memory.used --format=csv,noheader
    print(f"GPU: {gpu_info[0]}")

# to Keep track of the original directory
original_dir = os.getcwd()

PyTorch version: 2.6.0+cu124
CUDA available: True
GPU: Tesla T4, 15360 MiB, 2 MiB


In [ ]:
!pip install -q transformers>=4.36.0 bitsandbytes>=0.41.0 accelerate>=0.25.0 peft>=0.7.0
!pip install -q datasets>=2.14.0 sentencepiece huggingface_hub
!pip install -q xformers  # Additional memory optimization for Colab

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 15.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Created a directory for our outputs on Drive
!mkdir -p /content/drive/MyDrive/model_outputs

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import torch
import transformers
from datasets import Dataset
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
import torch.nn as nn
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, TaskType
from transformers import BitsAndBytesConfig, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from huggingface_hub import notebook_login
from IPython.display import display, Markdown

In [ ]:
df = pd.read_csv("/content/better_processed_data2.0.csv")
if 'Unnamed: 0' in df.columns:
    df.drop(['Unnamed: 0'], axis=1, inplace=True)
print(f"Dataset size: {len(df)} rows")
df.head()

Dataset size: 7749 rows


,context,input,output
0,NaN,A crowd gathers in the great plaza,"A crowd gathers in the great plaza of Karth, a..."
1,S. copyright on this publication was renewed.]...,Ann says she'll kick him cross-eyed if,Ann says she'll kick him cross-eyed if he star...
2,The winds blow Cappen off his feet as the ship...,It turns out that the northmen fear,It turns out that the northmen fear the southm...
3,The Doll's Playhouse Martha discovers a black ...,Martha grabs hold of the lid and,Martha grabs hold of the lid and squeezes down...
4,S. copyright on this publication was renewed. ...,"When he wakes up, he finds no","When he wakes up, he finds no knife, no rocket..."


In [ ]:
#  Definedd helper function for parameter counting
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params:,} || all params: {all_param:,} || trainable%: {100 * trainable_params / all_param:.2f}%"
    )

In [ ]:
# Set up quantization and load model (with Colab memory optimizations)
# Cleared cache before loading model
torch.cuda.empty_cache()

# Configureed quantization - use 4-bit to save more memory on Colab
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit instead of 8-bit as we are using colab
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,  # Double quantization for memory savings
    bnb_4bit_quant_type="nf4"  # Used normalized float 4 quantization
)

# Loaded model with quantization
model = AutoModelForCausalLM.from_pretrained(
    "bigscience/bloomz-3b",
    quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True,
    offload_folder="/content/offload",  # For potential offloading if needed
)

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloomz-3b")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.01G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/199 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [ ]:
# Prepared model for training
# Proper preparation for 4-bit training
model = prepare_model_for_kbit_training(model)

# Enabled gradient checkpointing
model.gradient_checkpointing_enable()
model.enable_input_require_grads()

# Casted outputs to float32
class CastOutputToFloat(nn.Sequential):
    def forward(self, x):
        return super().forward(x).to(torch.float32)

model.lm_head = CastOutputToFloat(model.lm_head)

In [ ]:
# Configured and applied LoRA
lora_config = LoraConfig(
    r=2,  # Lower rank for Colab
    lora_alpha=16,
    target_modules=["query_key_value"],  # BLOOM specific
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# Applied LoRA
model = get_peft_model(model, lora_config)
print_trainable_parameters(model)


trainable params: 614,400 || all params: 1,823,523,840 || trainable%: 0.03%


In [ ]:
# Created dataset and preprocessing function
dataset = Dataset.from_pandas(df)

def generate_prompt(context: str, input: str, output: str) -> str:
    """
    Format the data as per the required instruction template
    """
    prompt = f"### Context:\n{context}\n\n### Input:\n{input}\n\n### Output:\n{output}</s>"
    return prompt

In [ ]:
#  Preprocessed the dataset
# Processed smaller batches to avoid OOM
def preprocess_function(examples, max_samples_per_batch=10):
    """Apply tokenization to the generated prompts in smaller batches"""

    all_results = {'input_ids': [], 'attention_mask': []}

    for i in range(0, len(examples['context']), max_samples_per_batch):
        end_idx = min(i + max_samples_per_batch, len(examples['context']))

        batch_contexts = examples['context'][i:end_idx]
        batch_inputs = examples['input'][i:end_idx]
        batch_outputs = examples['output'][i:end_idx]

        prompts = [generate_prompt(context, input_text, output)
                  for context, input_text, output in zip(batch_contexts, batch_inputs, batch_outputs)]

        batch_results = tokenizer(prompts, padding="max_length", truncation=True, max_length=256, return_tensors="pt")

        all_results['input_ids'].extend(batch_results['input_ids'])
        all_results['attention_mask'].extend(batch_results['attention_mask'])

    return all_results

# Processed the dataset
print("Processing dataset...")
processed_dataset = dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset.column_names
)

Processing dataset...


Map:   0%|          | 0/7749 [00:00<?, ? examples/s]

In [ ]:
#Configured and started training (with Colab optimizations)
# Disabled caching for training
model.config.use_cache = False

# Defined training arguments (smaller batches, fewer steps for Colab)
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/model_outputs',  # Save to Drive
    per_device_train_batch_size=1,  # Smaller batch size for Colab
    gradient_accumulation_steps=8,  # More gradient accumulation to compensate
    warmup_steps=50,
    max_steps=100,
    learning_rate=1e-3,
    fp16=True,
    logging_steps=1,
    save_strategy="steps",
    save_steps=25,           # Save more frequently
    report_to="none",        # Disable wandb, tensorboard, etc.
    # Enable memory optimizations
    gradient_checkpointing=True,
    optim="adamw_torch_fused",  # Used fused optimizer for better performance
)

# Created trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

# Trained the model
print("Starting training...")
trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Starting training...


Step,Training Loss
1,3.726800
2,3.709400
3,3.795000
4,3.858400
5,3.705000
6,3.668500
7,3.666800
8,3.529100
9,3.195800
10,3.379700


TrainOutput(global_step=100, training_loss=3.01730651140213, metrics={'train_runtime': 620.6336, 'train_samples_per_second': 1.289, 'train_steps_per_second': 0.161, 'total_flos': 2901097316352000.0, 'train_loss': 3.01730651140213, 'epoch': 0.10323912762937153})

In [ ]:
# Saved the model adapters locally and to Hugging Face Hub
print("Saving model...")
# Saved to Drive first
model.save_pretrained("/content/drive/MyDrive/model_outputs/final_model")

Saving model...


In [ ]:

model.config.use_cache = True

In [ ]:
# Defined inference function with memory optimizations for Colab
def make_inference(context, input, max_tokens=50):  # Reduced token count for Colab
    """
    Generate completion based on context and input
    """
    # Formatted the prompt according to our training template
    prompt = f"### Context:\n{context}\n\n### Input:\n{input}\n\n### Output:\n"

    # Tokenized the input
    inputs = tokenizer(prompt, return_tensors='pt').to("cuda")

    # Cleared CUDA cache before generation
    torch.cuda.empty_cache()

    # Generatde text with memory efficient settings
    with torch.cuda.amp.autocast():
        output_tokens = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            num_beams=1,  # Disable beam search to save memory
        )

    # Decoded and display the result
    generated_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    display(Markdown(generated_text))
    return generated_text

In [ ]:
# Tested the model with example
context = "When I look out into the night sky, I cannot discern the outline of a lion in the constellation Leo. I can make out the Big Dipper, and, if the night is clear, theLittle Dipper. I am at a loss to make out much of a hunter in Orion or a fish in the constellation of Pisces, to say nothing of a charioteer in Auriga"
input_text = "I was just amazed"

print("Generating text...")
make_inference(context, input_text)

Generating text...


<ipython-input-16-7cc103b9eea3>:16: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


### Context:
When I look out into the night sky, I cannot discern the outline of a lion in the constellation Leo. I can make out the Big Dipper, and, if the night is clear, theLittle Dipper. I am at a loss to make out much of a hunter in Orion or a fish in the constellation of Pisces, to say nothing of a charioteer in Auriga

### Input:
I was just amazed

### Output:
I was just amazed at the variety of color in the sky. I can hardly believe that the sun has not set yet. It is a very dark night. The moon is full. The sky is as dark as night itself.

'### Context:\nWhen I look out into the night sky, I cannot discern the outline of a lion in the constellation Leo. I can make out the Big Dipper, and, if the night is clear, theLittle Dipper. I am at a loss to make out much of a hunter in Orion or a fish in the constellation of Pisces, to say nothing of a charioteer in Auriga\n\n### Input:\nI was just amazed\n\n### Output:\nI was just amazed at the variety of color in the sky. I can hardly believe that the sun has not set yet. It is a very dark night. The moon is full. The sky is as dark as night itself.'